<a href="https://colab.research.google.com/github/JOEL352006/Pdf-Summarizer/blob/main/PDF_Summarizer_using_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai --upgrade
!pip install streamlit
!pip install PyMuPDF
!pip install pyngrok


In [ ]:
from google.colab import files
uploaded = files.upload()

pdf_path = next(iter(uploaded))
print("✅ Uploaded PDF:", pdf_path)


In [ ]:
%%writefile app.py
import streamlit as st
import fitz  # PyMuPDF
import google.generativeai as genai
import time
from google.api_core.exceptions import ResourceExhausted

# === Replace with your Gemini API Key ===
GOOGLE_API_KEY = "YOUR_GEMINI_API_KEY"  # 🔑 Put your key here
genai.configure(api_key=GOOGLE_API_KEY)

# Function to extract text from PDF
def extract_pdf_text(pdf_file):
    doc = fitz.open(stream=pdf_file.read(), filetype="pdf")
    full_text = ""
    for page in doc:
        text = page.get_text().strip()
        full_text += text + "\n"
    return full_text

# Function to summarize PDF content
def summarize_text(text):
    model = genai.GenerativeModel('models/gemini-1.5-flash')
    prompt = f"""Summarize the following PDF content in detail:
\"\"\"
{text}
\"\"\"
Summary:"""

    try:
        response = model.generate_content(prompt)
        return response.text
    except ResourceExhausted:
        st.error("❌ Rate limit exceeded. Please wait and try again later.")
        return "Summary unavailable due to API limit."
    except Exception as e:
        st.error(f"❌ Error: {e}")
        return "Summary generation failed."

# Function to answer user questions strictly based on summary
def answer_question(summary, user_query):
    model = genai.GenerativeModel('models/gemini-1.5-flash')
    prompt = f"""
You are a PDF assistant. Answer strictly based on the following summary.
If the answer is not in this summary, reply: "I cannot find this information in the document."

Summary:
\"\"\"
{summary}
\"\"\"

Question: {user_query}
"""
    try:
        response = model.generate_content(prompt)
        return response.text
    except ResourceExhausted:
        st.error("❌ Rate limit exceeded. Please wait and try again later.")
        return "Answer unavailable due to API limit."
    except Exception as e:
        st.error(f"❌ Error: {e}")
        return "Answer generation failed."

# ==== Streamlit UI ====
st.set_page_config(page_title="PDF Summarizer + Q/A Chatbot", layout="wide")
st.title("📄 PDF Summarizer + Chatbot")

uploaded_pdf = st.file_uploader("Upload a PDF file", type="pdf")

if uploaded_pdf is not None:
    st.info("📄 Extracting PDF content...")
    pdf_text = extract_pdf_text(uploaded_pdf)
    st.success("✅ PDF content extracted!")

    if st.button("🔍 Generate Summary"):
        with st.spinner("Generating summary using Gemini..."):
            summary = summarize_text(pdf_text)
        st.subheader("📃 PDF Summary:")
        st.write(summary)

        # Store summary in session
        st.session_state['summary'] = summary

    if 'summary' in st.session_state:
        user_query = st.text_input("Ask a question based on the PDF summary:")
        if user_query:
            with st.spinner("Zap is thinking..."):
                answer = answer_question(st.session_state['summary'], user_query)
            st.subheader("🤖 Zap's Answer:")
            st.write(answer)
else:
    st.warning("⚠️ Please upload a PDF file to proceed.")


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501
